In [1]:
from snp_query_box.dsnp_transform import dsnpTransform

In [2]:
#for dev purpose
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = False

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=RfInE5wm4KtjlzsF4MbiGq5xVsINdy&access_type=offline&code_challenge=bEXMGcTht8NYE068mSbfK_YHnCnwqvwEuGcMwIHrN1w&code_challenge_method=S256


Credentials saved to file: [C:\Users\A845514\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "edp-prod-mdcrbi-starsbi" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this pr

In [4]:
import os
import pandas as pd
import numpy as np

import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta


from snp_query_box import DsnpHelperFunction, snpQueries, populDashQueries

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [5]:
print(f"User: {USERNAME} Started creating pull_utilization, pull_er_visit, pull_ip_visit_from_hedis and *_12month in the shared storage")

User: a845514 Started creating pull_utilization, pull_er_visit, pull_ip_visit_from_hedis and *_12month in the shared storage


In [6]:
start_time = time.time()

In [7]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
one_year_rolling_back_date_str = one_year_rolling_back_date.strftime('%Y-%m-%d')
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

pulled_data_date = last_month
comp_report_data_month = last_month
hra_report_data_month = last_month
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

2024-06-01 00:00:00
2025-01-01
2025-05-01
2025-05-31


In [8]:
one_year_rolling_back_date_str

'2024-06-01'

In [9]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files/2025-05-31
The folder already exist
Output files will sit in the folder above.


In [10]:
reporting_year=2025

In [11]:
# if transition time first month...
if today_str.split('-')[1] =='01':
    first_date_of_year = str(int(first_date_of_year.split('-')[0])-1) + \
        '-01-01'
    print('Trainsition time, first date of year is updated by previous year:' + first_date_of_year)
    #first_date_of_year = '2024-01-01'

In [12]:
#locate data dump folder
if new_pull:
    pull_utilization_12month = snpQueries.pull_utilization_bq(start_date=one_year_rolling_back_date_str, end_date=reporting_end_date, mbr_list=None)
    pull_er_visit_12month = snpQueries.pull_er_visit_bq(enroll_start_date=today_str, enroll_end_date=today_str, start_date=one_year_rolling_back_date_str, end_date=today_str)
    pull_ip_visit_12month = snpQueries.pull_ip_visit_from_hedis_bq(start_date=one_year_rolling_back_date_str, end_date=today_str)

    pull_utilization_12month.to_parquet(f"{output_path}/pull_utilization_12month.parquet")
    pull_er_visit_12month.to_parquet(f"{output_path}/pull_er_visit_12month.parquet")
    pull_ip_visit_12month.to_parquet(f"{output_path}/pull_ip_visit_12month.parquet")
else:
    try:
        pull_utilization_12month = pd.read_parquet(f"{output_path}/pull_utilization_12month.parquet")
        pull_er_visit_12month = pd.read_parquet(f"{output_path}/pull_er_visit_12month.parquet")
        pull_ip_visit_12month = pd.read_parquet(f"{output_path}/pull_ip_visit_12month.parquet")

    except:
        pull_utilization_12month = snpQueries.pull_utilization_bq(start_date=one_year_rolling_back_date_str, end_date=reporting_end_date, mbr_list=None)
        pull_er_visit_12month = snpQueries.pull_er_visit_bq(enroll_start_date=today_str, enroll_end_date=today_str, start_date=one_year_rolling_back_date_str, end_date=today_str)
        pull_ip_visit_12month = snpQueries.pull_ip_visit_from_hedis_bq(start_date=one_year_rolling_back_date_str, end_date=today_str)

        pull_utilization_12month.to_parquet(f"{output_path}/pull_utilization_12month.parquet")
        pull_er_visit_12month.to_parquet(f"{output_path}/pull_er_visit_12month.parquet")
        pull_ip_visit_12month.to_parquet(f"{output_path}/pull_ip_visit_12month.parquet")

In [13]:
#col_amt = [col for col in pull_utilization_12month.columns if col.endswith("paid_amt")]
#pull_utilization_12month[col_amt] = pull_utilization_12month[col_amt].astype(float).round(2)

In [14]:
pull_utilization_ytd = snpQueries.pull_utilization_bq(start_date=first_date_of_year, end_date=reporting_end_date)

In [15]:
pull_er_visit_ytd = pull_er_visit_12month[pull_er_visit_12month["SRV_START_DT"]>=np.datetime64(first_date_of_year)]
pull_ip_visit_ytd = pull_ip_visit_12month[pull_ip_visit_12month["AdmitDate"]>=np.datetime64(first_date_of_year)]

In [16]:
pull_utilization_12month["dsnp_member_id"] = pull_utilization_12month["src_member_id"].str.strip()
pull_utilization_ytd["dsnp_member_id"] = pull_utilization_ytd["src_member_id"].str.strip()

In [17]:
pull_ip_visit_12month["dsnp_member_id"] = pull_ip_visit_12month["SourceMemberID"].str.strip()
pull_ip_visit_ytd["dsnp_member_id"] = pull_ip_visit_ytd["SourceMemberID"].str.strip()

In [18]:
pull_er_visit_12month["dsnp_member_id"] = pull_er_visit_12month["Member_ID"].str.strip()
pull_er_visit_ytd["dsnp_member_id"] = pull_er_visit_ytd["Member_ID"].str.strip()
pull_er_visit_12month.rename(columns = {"SRV_START_DT": "srv_start_dt"}, inplace=True)
pull_er_visit_ytd.rename(columns = {"SRV_START_DT": "srv_start_dt"}, inplace=True)


In [19]:
#Prepare call log data, one year back
utilization_df_ytd = dsnpTransform.get_utilization(pull_utilization_ytd, pull_ip_visit_ytd, pull_er_visit_ytd)
utilization_df_12month = dsnpTransform.get_utilization(pull_utilization_12month, pull_ip_visit_12month, pull_er_visit_12month)

In [20]:
#save it for monthly aggregation
utilization_df_ytd.to_parquet(f"{output_path}/utilization_df_ytd.parquet")
utilization_df_12month.to_parquet(f"{output_path}/utilization_df_12month.parquet")

In [21]:
print("Process time -- %s seconds" % (time.time() - start_time))

Process time -- 5120.76958322525 seconds
